## Analysis notebook

Results given on the data when in the test folds

### Imports

In [ ]:
from bokeh.plotting import output_file, save
import json
import os
import pickle
from locpix_points.data_loading import datastruc
from locpix_points.scripts.generate_features import main as extract_features
from locpix_points.scripts.visualise import visualise_torch_geometric, visualise_parquet, load_file
from locpix_points.evaluation.featanalyse import (
    explain,
    generate_umap_embedding,
    visualise_umap_embedding,
    generate_pca_embedding,
    visualise_pca_embedding,
    visualise_explanation,
    k_means_fn,
    get_prediction,
    subgraph_eval,
    pgex_eval,
    attention_eval,
    test_ensemble_averaging,
    struc_analysis_prep,
)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
import umap
import yaml

### Functions

In [ ]:
def find_graph_path(project_directory, file_name, file_folder):
    """Visualise raw data
    
    Args:
        project_directory (string): Location of project directory
        file_name (string) : Name of file to image
        file_folder (string) : Which folder the file is in"""
    
    train_file_map_path = os.path.join(project_directory, f"{file_folder}/train/file_map.csv")
    val_file_map_path = os.path.join(project_directory, f"{file_folder}/val/file_map.csv")
    test_file_map_path = os.path.join(project_directory, f"{file_folder}/test/file_map.csv")
    
    train_file_map = pd.read_csv(train_file_map_path)
    val_file_map = pd.read_csv(val_file_map_path)
    test_file_map = pd.read_csv(test_file_map_path)
    
    train_out = train_file_map[train_file_map["file_name"] == file_name]
    val_out = val_file_map[val_file_map["file_name"] == file_name]
    test_out = test_file_map[test_file_map["file_name"] == file_name]
    
    if len(train_out) > 0:
        folder = "train"
        file_name = train_out["idx"].values[0]
    if len(val_out) > 0:
        folder = "val"
        file_name = val_out["idx"].values[0]
    if len(test_out) > 0:
        folder = "test"
        file_name = test_out["idx"].values[0]
    
    return os.path.join(project_directory, f"{file_folder}/{folder}/{file_name}.pt")

### Parameters

In [ ]:
project_directory = ".."
# load config
config_loc = os.path.join(project_directory, "config/featanalyse_nn.yaml")
with open(config_loc, "r") as ymlfile:
    config_nn = yaml.safe_load(ymlfile)
label_map = config_nn["label_map"]
file_map = "../../../../../maps/linked_files.csv"
model_type = config_nn["model"]
model_name = config_nn["model_name"]
model_config = config_nn[model_type]

In [ ]:
umap_n_neighbours = 20
umap_min_dist = 0.5
pca_n_components = 2
device = 'cuda'
n_repeats=25
fold = 0
interactive = False
colour_by = "response" # options: [response, prediction, wt, wt_response, patient, correct]

### [PATIENT] Load in patient information

In [ ]:
linked_files = pd.read_csv(file_map)
linked_files = linked_files.drop(columns=["file_name"])
linked_files = linked_files.rename(columns = {"trial_no": "patient"})
linked_files = linked_files.drop_duplicates()

# mutations column
mutations = ['kras1213_sr', 'kras61_sr', 'kras146_sr', 'nras1213_sr', 'nras61_sr', 'braf_sr']
linked_files['all_wt'] = linked_files[mutations].apply(lambda row: 'yes' if all(val in ['WT', 'W/T'] for val in row) else 'no', axis=1)

## Feature analysis

### <mark>Generate features for explanation</mark>

CAREFUL WHETHER TO RUN AGAIN AS WILL CHANGE BELOW RESULTS


In [ ]:
extract_features_q = input("Are you sure you want to extract features? (YES I AM SURE): ")
if extract_features_q == "YES I AM SURE":
    extract_features(["-i",project_directory,"-c",config_loc,"-r",f"{n_repeats}"])

In [ ]:
train_df_nn_loc = os.path.join(project_directory, "output/train_locs.csv")
train_df_nn_loc = pd.read_csv(train_df_nn_loc)

train_df_nn_cluster = os.path.join(project_directory, "output/train_clusters.csv")
train_df_nn_cluster = pd.read_csv(train_df_nn_cluster)

train_df_nn_fov = os.path.join(project_directory, "output/train_fovs.csv")
train_df_nn_fov = pd.read_csv(train_df_nn_fov)

val_df_nn_loc = os.path.join(project_directory, "output/val_locs.csv")
val_df_nn_loc = pd.read_csv(val_df_nn_loc)

val_df_nn_cluster = os.path.join(project_directory, "output/val_clusters.csv")
val_df_nn_cluster = pd.read_csv(val_df_nn_cluster)

val_df_nn_fov = os.path.join(project_directory, "output/val_fovs.csv")
val_df_nn_fov = pd.read_csv(val_df_nn_fov)

test_df_nn_loc = os.path.join(project_directory, "output/test_locs.csv")
test_df_nn_loc = pd.read_csv(test_df_nn_loc)

test_df_nn_cluster = os.path.join(project_directory, "output/test_clusters.csv")
test_df_nn_cluster = pd.read_csv(test_df_nn_cluster)

test_df_nn_fov = os.path.join(project_directory, "output/test_fovs.csv")
test_df_nn_fov = pd.read_csv(test_df_nn_fov)

In [ ]:
def prep_features(train_df, val_df, test_df, fold):

    # get features present in the dataframe
    not_features = ["type", "file_name", "fold", "prediction"]
    features = [x for x in train_df.columns.to_list() if x not in not_features]

    ############ WARNING ##############
    # Be careful, if analysing neural net features
    # Is this the number of features you expect
    # Did this task use manual features as well
    num_features = len(features)
    print("Num features: ", num_features)

    train_data_list = []
    test_data_list = []

    # -------------------------------------- # 

    # --- Note ----
    ## If aggregate by folds, then we see the folds are separated in 
    ## UMAP space.
    ## i.e. We DON'T SEE: patients or other groups close to each  
    ## other in UMAP space irrelevant of the folds.
    ## Instead, we see the folds separated.
    
    #for fold in range(5):

    train_data = train_df[train_df["fold"] == fold]
    val_data = val_df[val_df["fold"] == fold]
    test_data = test_df[test_df["fold"] == fold]
            
    # feature vector
    train_data = train_data[features].values
    val_data = val_data[features].values
    test_data = test_data[features].values

    # concatenate train and val
    train_data = np.concatenate([train_data, val_data])

    scaler = StandardScaler().fit(train_data)
    X_train = scaler.transform(train_data)
    X_test = scaler.transform(test_data)

    train_data_list.append(X_train)
    test_data_list.append(X_test)

    # ------------------------------------- #

    X_train = np.concatenate(train_data_list)
    X_test = np.concatenate(test_data_list)
        
    return X_train, X_test

_, X_test_nn_loc = prep_features(train_df_nn_loc, val_df_nn_loc, test_df_nn_loc, fold)
_, X_test_nn_cluster = prep_features(train_df_nn_cluster, val_df_nn_cluster, test_df_nn_cluster, fold)
_, X_test_nn_fov = prep_features(train_df_nn_fov, val_df_nn_fov, test_df_nn_fov, fold)

test_df_nn_loc = test_df_nn_loc[test_df_nn_loc["fold"] == fold]
test_df_nn_cluster = test_df_nn_cluster[test_df_nn_cluster["fold"] == fold]
test_df_nn_fov = test_df_nn_fov[test_df_nn_fov["fold"] == fold]

### [PATIENT] Incorporate patient information

In [ ]:
test_df_nn_loc["patient"] = test_df_nn_loc['file_name'].str.extract(r'_P(\d+)')
test_df_nn_loc = test_df_nn_loc.astype({'patient': 'int64'})
test_df_nn_loc = test_df_nn_loc.merge(linked_files, on="patient")

test_df_nn_cluster["patient"] = test_df_nn_cluster['file_name'].str.extract(r'_P(\d+)')
test_df_nn_cluster = test_df_nn_cluster.astype({'patient': 'int64'})
test_df_nn_cluster = test_df_nn_cluster.merge(linked_files, on="patient")

test_df_nn_fov["patient"] = test_df_nn_fov['file_name'].str.extract(r'_P(\d+)')
test_df_nn_fov = test_df_nn_fov.astype({'patient': 'int64'})
test_df_nn_fov = test_df_nn_fov.merge(linked_files, on="patient")

test_df_nn_loc['wt_response'] = test_df_nn_loc.apply(lambda row: 
    "wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'yes' else 
    "non_wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'no' else 
    "non_responder" if row['type'] == 'no_response' else None, axis=1)

test_df_nn_cluster['wt_response'] = test_df_nn_cluster.apply(lambda row: 
    "wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'yes' else 
    "non_wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'no' else 
    "non_responder" if row['type'] == 'no_response' else None, axis=1)

test_df_nn_fov['wt_response'] = test_df_nn_fov.apply(lambda row: 
    "wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'yes' else 
    "non_wt_responder" if row['type'] == 'any_response' and row['all_wt'] == 'no' else 
    "non_responder" if row['type'] == 'no_response' else None, axis=1)

#### UMAP

In [ ]:
test_umap_embedding_nn_loc_path = os.path.join(project_directory, f"output/test_umap_embedding_nn_loc_{fold}.pkl")
test_umap_embedding_nn_cluster_path = os.path.join(project_directory, f"output/test_umap_embedding_nn_cluster_{fold}.pkl")
test_umap_embedding_nn_fov_path = os.path.join(project_directory, f"output/test_umap_embedding_nn_fov_{fold}.pkl")

if not os.path.exists(test_umap_embedding_nn_loc_path):
    test_umap_embedding_nn_loc = generate_umap_embedding(X_test_nn_loc, umap_min_dist, umap_n_neighbours)
    with open(test_umap_embedding_nn_loc_path, "wb") as f:
        pickle.dump(test_umap_embedding_nn_loc, f)
    f.close()

if not os.path.exists(test_umap_embedding_nn_cluster_path):
    test_umap_embedding_nn_cluster = generate_umap_embedding(X_test_nn_cluster, umap_min_dist, umap_n_neighbours)
    with open(test_umap_embedding_nn_cluster_path, "wb") as f:
        pickle.dump(test_umap_embedding_nn_cluster, f)
    f.close()

if not os.path.exists(test_umap_embedding_nn_fov_path):
    test_umap_embedding_nn_fov = generate_umap_embedding(X_test_nn_fov, umap_min_dist, umap_n_neighbours)
    with open(test_umap_embedding_nn_fov_path, "wb") as f:
        pickle.dump(test_umap_embedding_nn_fov, f)
    f.close()

In [ ]:
print("------ LOC ENCODER -------")
with open(test_umap_embedding_nn_loc_path, "rb") as f:
    test_umap_embedding_nn_loc = pickle.load(f)
visualise_umap_embedding(test_umap_embedding_nn_loc, test_df_nn_loc, label_map, interactive=interactive, colour_by=colour_by)


In [ ]:
print("------ CLUSTER ENCODER -------")
with open(test_umap_embedding_nn_cluster_path, "rb") as f:
        test_umap_embedding_nn_cluster = pickle.load(f)
visualise_umap_embedding(test_umap_embedding_nn_cluster, test_df_nn_cluster, label_map, interactive=interactive, colour_by=colour_by)

In [ ]:
print("------ FOV ENCODER -------")
with open(test_umap_embedding_nn_fov_path, "rb") as f:
    test_umap_embedding_nn_fov = pickle.load(f)
plot = visualise_umap_embedding(test_umap_embedding_nn_fov, test_df_nn_fov, label_map, interactive=interactive, colour_by=colour_by)

## Structure analysis

### Prepare data for structure analysis

In [ ]:
struc_analysis_prep(project_directory, fold, False, model_type, model_name, model_config, n_repeats, device)

In [ ]:
# get item to evaluate on
file_name = None

#### Load in configuration

In [ ]:
gt_label_map = {int(val): key for key,val in config_nn["label_map"].items()}

#### Visualise raw file

In [ ]:
file_folder = "preprocessed/gt_label"
file_path = os.path.join(project_directory, file_folder, file_name + ".parquet")
spheres = False
sphere_size = 50
visualise_parquet(file_path, 'x', 'y', None, 'channel', {0: "channel_0", 1: "channel_1", 2: "channel_2", 3: "channel_3"}, cmap=['k'], spheres=spheres, sphere_size=sphere_size)

#### Visualise file

In [ ]:
file_folder = f"processed/fold_{fold}"
file_loc = find_graph_path(project_directory, file_name, file_folder)
visualise_torch_geometric(file_loc)

#### Load in cluster model

In [ ]:
cluster_model = torch.load(os.path.join(project_directory, f"output/homogeneous_dataset/cluster_model.pt"), weights_only=False)
cluster_model.to(device)
cluster_model.eval()

#### Load datasets

In [ ]:
cluster_train_folder = os.path.join(project_directory, f"output/homogeneous_dataset/fold_{fold}/train")
cluster_val_folder = os.path.join(project_directory, f"output/homogeneous_dataset/fold_{fold}/val")
cluster_test_folder = os.path.join(project_directory, f"output/homogeneous_dataset/fold_{fold}/test")

cluster_train_set = datastruc.ClusterDataset(
    None,
    cluster_train_folder,
    label_level=None,
    pre_filter=None,
    save_on_gpu=None,
    transform=None,
    pre_transform=None,
    fov_x=None,
    fov_y=None,
)

cluster_val_set = datastruc.ClusterDataset(
    None,
    cluster_val_folder,
    label_level=None,
    pre_filter=None,
    save_on_gpu=None,
    transform=None,
    pre_transform=None,
    fov_x=None,
    fov_y=None,
)

cluster_test_set = datastruc.ClusterDataset(
    None,
    cluster_test_folder,
    label_level=None,
    pre_filter=None,
    save_on_gpu=None,
    transform=None,
    pre_transform=None,
    fov_x=None,
    fov_y=None,
)

#### Get items to evaluate on 

In [ ]:
cluster_dataitem, prediction = get_prediction(
    file_name,
    cluster_model, 
    cluster_train_set, 
    cluster_val_set, 
    cluster_test_set, 
    project_directory,
    cluster_train_folder,
    cluster_val_folder,
    cluster_test_folder,
    device, 
    gt_label_map)

#### SubgraphX

In [ ]:
subgraph_config = {
    # number of iterations to get prediction
    "rollout":  100, # 20
    # number of atoms of leaf node in search tree
    "min_atoms": 5,
    # hyperparameter that encourages exploration
    "c_puct": 10.0,
    # number of atoms to expand when extend the child nodes in the search tree
    "expand_atoms": 14,
    # whether to expand the children nodes from high degreee to low degree when extend the child nodes in the search tree
    "high2low": False,
    # number of local radius to caclulate
    "local_radius": 4,
    # sampling time of montecarlo approxim
    "sample_num": 100, # 100
    # reward method
    "reward_method": "mc_l_shapley",
    # subgrpah building method
    "subgraph_building_method": "split",
    # maximum number of nodes to include in subgraph when generating explanation
    "max_nodes": None,
    # number of classes
    "num_classes": None,
}

In [ ]:
 # ---- subgraphx -----
_, _, cluster_dataitem, node_imp = subgraph_eval(cluster_model, device, subgraph_config, cluster_dataitem, prediction)
    

In [ ]:
# visualise overlaid subgraph

file_folder = f"processed/fold_{fold}"
file_loc = find_graph_path(project_directory, file_name, file_folder)

visualise_explanation(
    cluster_dataitem.pos,
    cluster_dataitem.edge_index,
    node_imp=node_imp.to(device),
    edge_imp=None,
    overlay=True,
    file_loc=file_loc, 
)

#### Attention

In [ ]:
attention_config = {
    #  scale: cluster
    "scale": "cluster",
    #  # how to combine attention scores across multiple attention heads
    "reduce": "max",
    # threshold to apply to edge mask for pyg explain
    "edge_mask_threshold": 0.5,
}

In [ ]:
# ---- attention -----
# use model - logprobs or clustermodel - raw
attention_eval(cluster_model, attention_config, cluster_dataitem, device)

### Statistical tests

# Legacy

#### PCA

In [ ]:
train_pca_embedding_nn_loc = generate_pca_embedding(X_train_nn_loc, pca_n_components)
train_pca_embedding_nn_fov = generate_pca_embedding(X_train_nn_fov, pca_n_components)
train_pca_embedding_nn_cluster = generate_pca_embedding(X_train_nn_cluster, pca_n_components)
if final_test:
    test_pca_embedding_nn_loc = generate_pca_embedding(X_test_nn_loc, pca_n_components)
    test_pca_embedding_nn_fov = generate_pca_embedding(X_test_nn_fov, pca_n_components)
    test_pca_embedding_nn_cluster = generate_pca_embedding(X_test_nn_cluster, pca_n_components)

In [ ]:
print("------ LOC ENCODER -------")
visualise_pca_embedding(train_pca_embedding_nn_loc, train_df_nn_loc, label_map)
if final_test:
    visualise_pca_embedding(test_pca_embedding_nn_loc, test_df_nn_loc, label_map)

In [ ]:
print("------ CLUSTER ENCODER -------")
visualise_pca_embedding(train_pca_embedding_nn_cluster, train_df_nn_cluster, label_map)
if final_test:
    visualise_pca_embedding(test_pca_embedding_nn_cluster, test_df_nn_cluster, label_map)


In [ ]:
print("------ FOV ENCODER -------")
visualise_pca_embedding(train_pca_embedding_nn_fov, train_df_nn_fov, label_map)
if final_test:
    plot = visualise_pca_embedding(test_pca_embedding_nn_fov, test_df_nn_fov, label_map)
    if False:
        output_file_loc  = os.path.join(project_directory, "output", "nn_fov.html")
        output_file(output_file_loc)
        save(plot)

#### K-means

In [ ]:
print("----- LOC ------")
k_means_fn(X_train_nn_loc, train_df_nn_loc, label_map)
if final_test:
    k_means_fn(X_test_nn_loc, test_df_nn_loc, label_map)

print("----- CLUSTER ------")
k_means_fn(X_train_nn_cluster, train_df_nn_cluster, label_map)
if final_test:
    k_means_fn(X_test_nn_cluster, test_df_nn_cluster, label_map)

print("----- FOV ------")
k_means_fn(X_train_nn_fov, train_df_nn_fov, label_map)
if final_test:
    k_means_fn(X_test_nn_fov, test_df_nn_fov, label_map)


#### PgEx

In [ ]:
pgex_config = {
    # threshold to apply to edge mask for pyg explain
    "edge_mask_threshold": 0.5,
}

In [ ]:
# ---- pgexplainer ----
pg_explainer = torch.load(os.path.join(project_directory, f"output/pg_explainer.pt"), weights_only=False) 
pgex_eval(cluster_model, pg_explainer, cluster_dataitem, device, pgex_config)